In [5]:
import requests
import parsel

In [162]:
asc_url = 'http://www.npr.org/sections/allsongs/163479981/our-show'
response = requests.get(asc_url)
sel = parsel.Selector(text=response.content.decode('utf-8'))

In [161]:
links = sel.xpath('//h1/a').extract()

In [164]:
show_url = 'http://www.npr.org/sections/allsongs/2016/02/09/466146044/new-mix-shearwater-lily-madeleine-eskimeaux-more'
content = response.content.decode('utf-8')
showsel = parsel.Selector(content)

In [163]:
showsel.xpath('//*[@class="audio listen"]/@data-player')

[<Selector xpath='//*[@class="audio listen"]/@data-player' data='{"storyId":"466146044","mediaId":"466147'>,
 <Selector xpath='//*[@class="audio listen"]/@data-player' data='{"storyId":"466146044","mediaId":"466147'>,
 <Selector xpath='//*[@class="audio listen"]/@data-player' data='{"storyId":"466146044","mediaId":"466147'>,
 <Selector xpath='//*[@class="audio listen"]/@data-player' data='{"storyId":"466146044","mediaId":"466173'>,
 <Selector xpath='//*[@class="audio listen"]/@data-player' data='{"storyId":"466146044","mediaId":"466147'>,
 <Selector xpath='//*[@class="audio listen"]/@data-player' data='{"storyId":"466146044","mediaId":"466147'>,
 <Selector xpath='//*[@class="audio listen"]/@data-player' data='{"storyId":"466146044","mediaId":"466147'>,
 <Selector xpath='//*[@class="audio listen"]/@data-player' data='{"storyId":"466146044","mediaId":"466147'>]

In [165]:
import parsel
import requests

class Show(object):
    def __init__(self, url):
        self.url = url
        self.response = requests.get(url, headers={
            'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 9_1 like Mac OS X) '
                          'AppleWebKit/601.1.46 (KHTML, like Gecko) '
                          'Version/9.0 Mobile/13B143 Safari/601.1',
        })
        self.sel = parsel.Selector(self.response.content.decode('utf-8'))
        
        self.extract_timestamp()
        self.count_songs()
    
    def extract_timestamp(self):
        base_url = self.sel.xpath('//a[@class="download"]/@href').extract_first()
        assert 'asc_wholeshow' in base_url
        x = base_url.find('npr/asc/') + len('npr/asc/')
        y = base_url.find('_asc_')
        self.url_timestamp = base_url[x:y]
        
    def song_url(self, n):
        return ('http://public.npr.org/anon.npr-mp3/npr/asc/{}_asc_{:02d}.mp3'
                .format(self.url_timestamp, n))
    
    def count_songs(self):
        self.n_songs = len(self.sel.xpath('//*[@class="audio listen"]/@data-player'))
        
    def song_urls(self):
        for n in range(1, self.n_songs + 1):
            yield self.song_url(n)

In [166]:
show = Show(show_url)

In [168]:
for url in show.song_urls():
    response = requests.get(url)
    save_as = url[url.rfind('/')+1:]
    with open(save_as, 'wb') as f:
        f.write(response.content)

In [ ]:
bad_shows = [
    'http://www.npr.org/sections/allsongs/2016/02/05/465551231/all-songs-1-andrew-bird-gets-personal',
    'http://www.npr.org/sections/allsongs/2016/01/15/463163048/all-songs-1-glen-hansard-plays-our-sweet-16-celebration',
]
good_shows = [
    'http://www.npr.org/sections/allsongs/2016/01/26/463484599/our-top-discoveries-at-globalfest-2016',
    'http://www.npr.org/sections/allsongs/2016/02/09/466146044/new-mix-shearwater-lily-madeleine-eskimeaux-more',
]

In [ ]:
def 